In [15]:
from cobra import Metabolite
from cobra import Reaction
import cameo
import cobra

In [16]:
#importing iJO1366 model
model = cameo.models.bigg.iJO1366
model.reactions.EX_co2_e.lower_bound = 0

#metabolite declaration
lactate = model.metabolites.get_by_id('lac__L_c')
accoa = model.metabolites.get_by_id('accoa_c')
lactyl_coa = Metabolite('laccoa_c', formula = 'C24H36N7O18P3S', name = 'lactyl-CoA', compartment = 'c', charge = -4)
ac = model.metabolites.get_by_id('ac_c')
coa = model.metabolites.get_by_id('coa_c')
polylactic_acid = Metabolite('polylactic_c', formula = 'C3H4O2', name = 'polylactic acid', compartment = 'c')


In [17]:
#reactions being imported into iJO1366 model

#reaction 1: lactate + accoa <-> lactyl_coa + ac
reaction1 = Reaction('lactate_laccoa')
reaction1.name = 'lactate to lactyl-CoA'
reaction1.lower_bound = -1000
reaction1.upper_bound = 1000
reaction1.add_metabolites({lactate: -1.0, accoa: -1.0, lactyl_coa: 1.0, ac: 1.0})
model.add_reaction(reaction1)

#reaction 2: lactyl_coa <-> polylactic_acid + coa
reaction2 = Reaction('laccoa_polylactic')
reaction2.name = 'lactyl-CoA to polylactic acid'
reaction2.lower_bound = -1000
reaction2.upper_bound = 1000
reaction2.add_metabolites({lactyl_coa: -1.0, polylactic_acid: 1.0, coa: 1.0})
model.add_reaction(reaction2)

#reaction 3: polylactic_acid ->
reaction3 = Reaction('EX_polylactic_e')
reaction3.name = 'polylactic acid consumption'
reaction3.lower_bound = 0
reaction3.upper_bound = 1000
reaction3.add_metabolites({polylactic_acid: -1.0})
model.add_reaction(reaction3)


In [18]:
#optimum flux calculation
model.objective = 'EX_polylactic_e'
max_flux = 0
max_flux=(model.optimize(objective_sense='maximize').objective_value)
print(max_flux)

16.462328767123285


In [19]:
model.summary()

In [20]:
for i in range(-1,-4,-1):
    if model.reactions[i].check_mass_balance() == {}:
        pass
    else:
        print(model.reactions[i])

EX_polylactic_e: polylactic_c --> 


In [21]:
cobra.io.write_sbml_model(model, "polylactic_acid.xml")